# Time offset when exporting naively from Mixxx to Rekordbox

## Be Careful

Do not run this notebook as I did not push the test audio files in the repo.

## Context

The grids of some tracks display an offset once imported into Rekordbox.

This problem is known. In [this comment](https://github.com/mixxxdj/mixxx/pull/2119#issuecomment-531575124) we can understand the solution:

```
if mp3 does NOT have a Xing/INFO tag:
     case = 'A'
     correction = 0ms
 
 elif mp3 has Xing/INFO, but does NOT have a LAME tag:
     # typical case: has LAVC header instead
     case = 'B'
     correction = 26ms
 
 elif LAME tag has invalid CRC:
     # typical case: CRC= zero
     case = 'C'
     correction = 26ms
     
 elif LAME tag has valid CRC:
     case = 'D'
     correction = 0ms
```

In [this comment](https://github.com/mixxxdj/mixxx/pull/2119#issuecomment-533952875) we can understand how it works when taking into consideration the decoder used by Mixxx:

```
if MAD decoder
	if case A or D
		offset = -26
if CoreAudio decoder
	if case A
		offset = -13
	if case B
		offset = -11
	if case C
		offset = -26
	if case D
		offset = -50
if FFmpeg4.1 decoder
	if case D
		offset = -26
```


And here's [how it's implement in the code](https://github.com/mixxxdj/mixxx/blob/8f647908af460e53fe8a860b6ce0964a93a55112/src/library/rekordbox/rekordboxfeature.cpp#L1261).

In the discussion @pestrela talked about using EyeD3 only (instead of also relaying on mp3guessenc). We can have a glimpse on how he does it [in this code](https://github.com/pestrela/music/blob/master/traktor/26ms_offsets/bin/mp3_check_encoder.sh) (there's some inadvertently pasted code at the end…).

Usng EyeD3 alone would be very convenient for this project as EyeD3 installation is simple (uses pip). This NoteBook is here te check taht it's indeed possible.

## MP3 files

I will use the test files from @pestrela repository: https://github.com/pestrela/music/tree/master/traktor/26ms_offsets/examples_tagged

In [1]:
from pathlib import Path

mp3_files = [
    Path("grid_offset_problems--mp3/case_a__airbase_fall__no_xing_only_lame.mp3"),
    Path("grid_offset_problems--mp3/case_a__armin_Nehalennia__nothing.mp3"),
    Path("grid_offset_problems--mp3/case_b__adele_paul_damixie__xing_lavc.mp3"),
    Path("grid_offset_problems--mp3/case_b__avicci_tim_berg__info_lavc.mp3"),
    Path(
        "grid_offset_problems--mp3/case_b__david_guetta_memories__xing_lame_on_second_frame.mp3"
    ),
    Path("grid_offset_problems--mp3/case_b__diogo_dialeto__xng_lavc.mp3"),
    Path("grid_offset_problems--mp3/case_b__estiva__info_lavf.mp3"),
    Path("grid_offset_problems--mp3/case_c__factorb_luna__crc_zero.mp3"),
    Path("grid_offset_problems--mp3/case_d__pobsky__crc_ok.mp3"),
]

[PosixPath('grid_offset_problems--mp3/case_a__airbase_fall__no_xing_only_lame.mp3'),
 PosixPath('grid_offset_problems--mp3/case_a__armin_Nehalennia__nothing.mp3'),
 PosixPath('grid_offset_problems--mp3/case_b__adele_paul_damixie__xing_lavc.mp3'),
 PosixPath('grid_offset_problems--mp3/case_b__avicci_tim_berg__info_lavc.mp3'),
 PosixPath('grid_offset_problems--mp3/case_b__david_guetta_memories__xing_lame_on_second_frame.mp3'),
 PosixPath('grid_offset_problems--mp3/case_b__diogo_dialeto__xng_lavc.mp3'),
 PosixPath('grid_offset_problems--mp3/case_b__estiva__info_lavf.mp3'),
 PosixPath('grid_offset_problems--mp3/case_c__factorb_luna__crc_zero.mp3'),
 PosixPath('grid_offset_problems--mp3/case_d__pobsky__crc_ok.mp3')]

## Using EyeD3

In [2]:
%pip install eyed3

Note: you may need to restart the kernel to use updated packages.


In [4]:
import eyed3, eyed3.mp3


def has_xing_info(audiofile: eyed3.mp3.Mp3AudioFile) -> bool:
    return audiofile.info.xing_header is not None


def has_lame_tag(audiofile: eyed3.mp3.Mp3AudioFile) -> bool:
    return len(audiofile.info.lame_tag) > 0


def has_valid_CRC_tag(audiofile: eyed3.mp3.Mp3AudioFile) -> bool:
    try:
        return audiofile.info.lame_tag["music_crc"] > 0
    except KeyError:
        return False


def find_case(audiofile: eyed3.mp3.Mp3AudioFile) -> str:
    if not has_xing_info(audiofile):
        return "A"
    elif not has_lame_tag(audiofile):
        return "B"
    elif not has_valid_CRC_tag(audiofile):
        return "C"
    else:
        return "D"


for file in mp3_files:
    audiofile = eyed3.load(file)
    print(
        file.stem,
        ":\n\t",
        find_case(audiofile),
        has_xing_info(audiofile),
        has_lame_tag(audiofile),
        has_valid_CRC_tag(audiofile),
    )

Lame tag CRC check failed
Lame tag CRC check failed


case_a__airbase_fall__no_xing_only_lame :
	 A False True True
case_a__armin_Nehalennia__nothing :
	 A False False False
case_b__adele_paul_damixie__xing_lavc :
	 B True False False
case_b__avicci_tim_berg__info_lavc :
	 B True False False
case_b__david_guetta_memories__xing_lame_on_second_frame :
	 B True False False
case_b__diogo_dialeto__xng_lavc :
	 B True False False
case_b__estiva__info_lavf :
	 B True False False
case_c__factorb_luna__crc_zero :
	 C True True False
case_d__pobsky__crc_ok :
	 D True True True


It looks good!

Let's do an extra check with my files that I am not going to share :-p

In [5]:
new_mp3_files = [
    Path("/media/francois/MEGAMIX/DNB/Sleepnet - Lapse/01 - Lapse.mp3"),
    Path("/media/francois/MEGAMIX/DNB/Buunshin - All About This/01 - Acolyte.mp3"),
    Path(
        "/media/francois/MEGAMIX/DNB/VRAC/DJ Rush - Motherfucking Bass (Phace Bootleg).mp3"
    ),
    Path("/media/francois/MEGAMIX/DNB/Noisia - Purpose EP/05 - Asteroids.mp3"),
]

In [6]:
for file in new_mp3_files:
    audiofile = eyed3.load(file)
    print(
        file.stem,
        ":\n\t",
        find_case(audiofile),
        has_xing_info(audiofile),
        has_lame_tag(audiofile),
        has_valid_CRC_tag(audiofile),
    )

01 - Lapse :
	 D True True True
01 - Acolyte :
	 D True True True
DJ Rush - Motherfucking Bass (Phace Bootleg) :
	 B True False False
05 - Asteroids :
	 B True False False


## Checking my module

In [7]:
from encoder_tools import get_offset_ms

In [8]:
for file in new_mp3_files:
    audiofile = eyed3.load(file)
    print(file.stem, ":", get_offset_ms(file, mp3_decoder="MAD"))

01 - Lapse : 26
01 - Acolyte : 26
DJ Rush - Motherfucking Bass (Phace Bootleg) : 0
05 - Asteroids : 0
